In [84]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import csv

In [95]:
data_df = pd.read_csv('./Steam.csv', sep=',', names=["User ID","Name","Type","Hours"], engine='python')
#data_df = data_df[data_df["Type"] == "purchase"]


unique_GameID = data_df['Name'].unique()
unique_UserID = data_df['User ID'].unique()

user_count_j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = user_count_j
    user_count_j += 1

game_count_j = 0
game_old2new_id_dict = dict()
for i in unique_GameID:
    game_old2new_id_dict[i] = game_count_j
    game_count_j += 1

# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
for j in range(len(data_df)):
    data_df.at[j, 'User ID'] = user_old2new_id_dict[data_df.at[j, 'User ID']]
   
    
    data_df.at[j, 'Name'] = game_old2new_id_dict[data_df.at[j, 'Name']]

data_df = data_df[data_df["Type"] == "purchase"]

#import survey data:

with open("GGs 489 survey.csv", newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

data = data[1:]

for i in range (len(data)):
    data[i][1] = data[i][1].split(";")

#Collect the survey data into survey_df
survey_data = []

for i in range (len(data)):
    
    user_old2new_id_dict[data[i][2]] = user_count_j
    user_count_j += 1
    for k in data[i][1]:
        survey_data.append([user_old2new_id_dict[data[i][2]],game_old2new_id_dict[k],"purchase",1.0])

survey_df = pd.DataFrame(survey_data,columns=["User ID","Name","Type","Hours"])

#attach the survey data to the given data
data_df = data_df.append(survey_df,ignore_index=True)


In [96]:
test_df = data_df.sample(frac = 0.3, random_state = np.random.randint(1024))
train_df = data_df.drop(test_df.index)

print ("The testing dataframe is of size: " + str(len(test_df)))
print ("The training dataframe is of size: " + str(len(train_df)))

The testing dataframe is of size: 38859
The training dataframe is of size: 90670


In [94]:
num_user = len(data_df['User ID'].unique())
num_game = len(data_df['Name'].unique())

data_mat = coo_matrix((data_df['Hours'].values, (data_df['User ID'].values, data_df['Name'].values)), shape=(num_user, num_game)).toarray().astype(float)
train_mat = coo_matrix((train_df['Hours'].values, (train_df['User ID'].values, train_df['Name'].values)), shape=(num_user, num_game)).toarray().astype(float)
test_mat = coo_matrix((test_df['Hours'].values, (test_df['User ID'].values, test_df['Name'].values)), shape=(num_user, num_game)).toarray().astype(float)

In [111]:
#in order to extract user id and games from matrix use inverse dictionaries:
inv_names = {v: k for k, v in game_old2new_id_dict.items()}
inv_users = {v: k for k, v in user_old2new_id_dict.items()}

usertest = 12394
gametest = 0
print ("User: " + str(inv_users[usertest]) + " has value "+ str(data_mat[usertest,gametest]) + " for game: "+ str(inv_names[gametest]))

User: Markus has value 1.0 for game: The Elder Scrolls V Skyrim
